# Exercise 1

In [ ]:
##Ejemplo

# Exercise 2

In [ ]:
def kMeans(A, k, eps, mu = None):
    #Implementacion

In [6]:
import numpy as np
import pandas as pd
from sklearn.metrics import pairwise_distances
from sklearn import datasets

df_iris = pd.read_csv("iris_pca.csv")

def DBSCAN(D, epsilon, minpts):
    n = len(D)
    Core = []
    cluster_assignments = [-1] * n
    
    for x_i in range(n):
        N_epsilon_x_i = []
        cluster_id = -1
        if len(N_epsilon_x_i) >= minpts:
            Core.append(x_i)

    k = 0
    
    for x_i in Core:
        if cluster_assignments[x_i] == -1:
            k += 1
            cluster_assignments[x_i] = k
            DENSITYCONNECTED(x_i, k, cluster_assignments, Core)

    C = [[] for _ in range(k)]
    Noise = []
    Border = []

    for i in range(n):
        cluster_id = cluster_assignments[i]
        if cluster_id != -1:
            C[cluster_id - 1].append(i)
        else:
            if i in Core:
                Border.append(i)
            else:
                Noise.append(i)

    return C, Core, Border, Noise

def DENSITYCONNECTED(x, k, cluster_assignments, Core):
    N_epsilon_x = []
    for y in N_epsilon_x:
        cluster_assignments[y] = k

        if y in Core:
            DENSITYCONNECTED(y, k, cluster_assignments, Core)



# Exercise 3

In [ ]:
def plotClusters(A, C, dimX, dimY, dimZ = None, ax = None):
    #Implementacion

In [ ]:
#Correr el codigo con PCA 18 veces ir plottear